In [26]:
#Importing libraries
import pandas as pd
from sqlalchemy import create_engine

In [27]:
#Connection with SQL
db_config = {'user': 'postgres',         # user name
             'pwd': '1234', # password
             'host': 'localhost',
             'port': 5432,              # connection port
             'db': 'My_Projects'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string)

In [28]:
#Netflix table view
query = 'SELECT * FROM netflix'

netflix = pd.io.sql.read_sql(query, con = engine)
netflix.head(10)

,title,rating,ratinglevel,ratingdescription,release year,user rating score,user rating size
0,White Chicks,PG-13,"crude and sexual humor, language and some drug...",80,2004,82,80
1,Lucky Number Slevin,R,"strong violence, sexual content and adult lang...",100,2006,NA,82
2,Grey's Anatomy,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2016,98,80
3,Prison Break,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2008,98,80
4,How I Met Your Mother,TV-PG,Parental guidance suggested. May not be suitab...,70,2014,94,80
5,Supernatural,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2016,95,80
6,Breaking Bad,TV-MA,For mature audiences. May not be suitable for...,110,2013,97,80
7,The Vampire Diaries,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2017,91,80
8,The Walking Dead,TV-MA,For mature audiences. May not be suitable for...,110,2015,98,80
9,Pretty Little Liars,TV-14,Parents strongly cautioned. May be unsuitable ...,90,2016,96,80


## Query 1

In [30]:
#All shows with 'dead' titles
query = "select title from netflix where title ilike any(array['%%dead','dead%%','%%dead%%'])"

q1 = pd.io.sql.read_sql(query, con = engine)
q1

,title
0,The Walking Dead
1,The Seven Deadly Sins
2,The Walking Dead
3,Drop Dead Diva
4,Drop Dead Diva


## Query 2

In [32]:
#All R-rating shows on netflix
query = '''select COUNT(title),rating from netflix
where rating = 'R'
group by rating;'''

q2 = pd.io.sql.read_sql(query, con = engine)
q2

,count,rating
0,19,R


## Query 3

In [34]:
#Shows not suitable for childrens 
query = '''select rating_sub,count(distinct title) from( 
select title,
CASE when ratinglevel ilike '%%suitable for children%%' then replace(right(ratinglevel,13),'.','') 
	  when ratinglevel ilike '%%suitable for all children%%' then 'Not for children'
	  else null end as rating_sub 
from netflix ) as rating_shows
GROUP BY rating_sub;'''

q3 = pd.io.sql.read_sql(query, con = engine)
q3

,rating_sub,count
0,14 and under,100
1,17 and under,60
2,asy violence,25
3,for children,12
4,Not for children,31
5,s 7 and older,23
6,None,249


## Query 4

In [36]:
#No. of shows released for Mature audience and childrens
query = '''select distinct(age_category),"release year" as year_of_release,count(title) as Show_count from
(select title,ratinglevel,"release year",
Case when ratinglevel ilike 'Suitable for children%%' then 'Children'
when ratinglevel ilike 'Suitable for all ages.' then 'everyone'
else 'Mature'
end as age_category
from netflix) as a
group by 1,2
order by 1,2;'''

q4 = pd.io.sql.read_sql(query, con = engine)
q4

,age_category,year_of_release,show_count
0,Children,1989,1
1,Children,1990,4
2,Children,1996,1
3,Children,1997,5
4,Children,1998,2
...,...,...,...
63,Mature,2013,37
64,Mature,2014,48
65,Mature,2015,74
66,Mature,2016,254


## Query 5

In [37]:
#Best user-rated action shows
query = '''select distinct(title),case when "user rating score" = 'NA' then null else "user rating score" end as "user rating score",ratinglevel from netflix
where ratinglevel ilike '%%action%%'
order by 2 desc
nulls last
limit 10;'''

q5 = pd.io.sql.read_sql(query, con = engine)
q5

,title,user rating score,ratinglevel
0,The BFG,97,"for action/peril, some scary moments and brief..."
1,The Jungle Book,97,for some sequences of scary action and peril
2,Zootopia,97,"some thematic elements, rude humor and action"
3,Alice Through the Looking Glass,96,fantasy action/peril and some language
4,Atlantis: The Lost Empire,96,action violence
5,Kubo and the Two Strings,96,"thematic elements, scary images, action and peril"
6,Minions,96,action and rude humor
7,Lilo & Stitch,95,mild sci-fi action
8,Home,94,mild action and some rude humor
9,Hotel Transylvania 2,93,"some scary images, action and rude humor"


## Query 6

In [38]:
#Movie contribution percentage per year
query = '''with movie_count as (
select count(distinct title) as numb,"release year" from netflix
group by 2
order by 1 desc
)
, percentage as (
	select "release year", round(numb / sum(numb) over (),2) *100  as percentage_of_total
	from movie_count
)
select * from percentage'''

q6 = pd.io.sql.read_sql(query, con = engine)
q6

,release year,percentage_of_total
0,2016,29.0
1,2015,12.0
2,2017,7.0
3,2014,7.0
4,2013,7.0
5,2012,5.0
6,2011,3.0
7,2010,3.0
8,2003,2.0
9,2008,2.0


## Query 7

In [39]:
#Best user-rated comedy movies
query = '''select distinct(title),case when "user rating score" = 'NA' then null else "user rating score" end as "user rating score",ratinglevel from netflix
where ratinglevel ilike '%%humor%%'
order by 2 desc
nulls last
limit 10;'''

q7 = pd.io.sql.read_sql(query, con = engine)
q7

,title,user rating score,ratinglevel
0,The BFG,97,"for action/peril, some scary moments and brief..."
1,Zootopia,97,"some thematic elements, rude humor and action"
2,Minions,96,action and rude humor
3,Coraline,95,"thematic elements, scary images, some language..."
4,Bee Movie,94,mild suggestive humor
5,Home,94,mild action and some rude humor
6,Hotel Transylvania 2,93,"some scary images, action and rude humor"
7,Kung Fu Panda 3,92,martial arts action and some mild rude humor
8,The Angry Birds Movie,92,for rude humor and action
9,The Pacifier,91,"action violence, language and rude humor"
